## Crawler to get info from Trovit

Install Libraries

In [2]:
!pip install selenium

In [216]:
import pandas as pd
import numpy
import time
from random import randint
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException

In [144]:
#driver=webdriver.Chrome('/Users/pablogoya/Documents/Master DS/chromedriver')

!pip install webdriver-manager
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.utils import ChromeType

driver = webdriver.Chrome(ChromeDriverManager(chrome_type=ChromeType.GOOGLE).install())

driver.get('https://casas.trovit.es/')

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/pablogoya/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


Defining scrapper function for Trovit

In [228]:
#Scrapping the Trovit website with multiple for loops

def trvt_scrapper(min_size,max_size,pages):
    
    #Cleaning lists:
 
    list_prices=[]
    list_features=[]
    list_locations=[]

    list_prices_full=[]
    list_features_full=[]
    list_locations_full=[]

#Scrapper with multiple for loops over the Trovit website

#First loop goes through each page and adds up the data to be able to build the DataFrame
    
    for u in range (1,pages+1):

        url1='https://casas.trovit.es/index.php/cod.search_homes/type.1/what_d.madrid/origin.2/price_min.100000/rooms_min.1/photo.1/area_min.'
        url2='/area_max.'
        url3='/bathrooms_min.1/order_by.relevance/region.Madrid/resultsPerPage.25/isUserSearch.1/page.'
        driver.get(url+str(min_size)+url2+str(max_size)+url3+str(u))
        
    #This is just to avoid robot blocking your IP
        time.sleep(randint(1,5))

    #This loop scrolls down automatically to replicate users browsing behaviour

        for i in range (randint(8,13)):

            html_txt=driver.page_source
            soup=BeautifulSoup(html_txt)
        
        #Create the BeautifulSoup searches

            locations=soup.find_all(attrs={'itemprop': 'address'})
            prices=soup.find_all('span',class_="amount")  
            features=soup.find_all('div' ,class_="property")
            
            list_prices=[]
            list_locations=[]
            list_features=[]

        #Search across the page for these parameters
        
            for price in prices:
                title1=price.getText()
                list_prices.append(title1)
            for feature in features:
                title2=feature.getText()
                list_features.append(title2)
            for location in locations:
                title3=location.getText()
                list_locations.append(title3)

            ActionChains(driver).key_down(Keys.PAGE_DOWN).key_up(Keys.PAGE_DOWN).perform()
            time.sleep(randint(1,10)/10)

    #Put all the data together, one page per loop, to pass it to the next step and build the dataset

        list_prices_full=list_prices_full+list_prices
        list_features_full=list_features_full+list_features
        list_locations_full=list_locations_full+list_locations

    
#Building the full DataFrame with all read data through the scrapper

    list_locations_full= [i.split(',', 1)[0] for i in list_locations_full]
    list_locations_full= [x.strip(' ') for x in list_locations_full]

    list_features_full_splitted = [list_features_full[i*3:(i+1)*3] for i in range((len(list_features_full)+3-1)//3)] 
    features_df= pd.DataFrame(list_features_full_splitted)
    features_df.columns= ['rooms','bathrooms','size']

    houses_list_full=[list_prices_full,list_locations_full]
    result_trvt_df=pd.DataFrame()
    result_trvt_df=pd.DataFrame (houses_list_full).transpose()
    result_trvt_df.columns=['prices','post_code']
    result_trvt_df= pd.concat([result_trvt_df, features_df], axis=1)   
              

    return result_trvt_df 


In [229]:
df_mad_40_60=trvt_scrapper(40,60,100)
time.sleep(randint(60,120))

In [230]:
df_mad_60_80=trvt_scrapper(60,80,100)
time.sleep(randint(60,120))

In [231]:
df_mad_80_100=trvt_scrapper(80,100,100)

In [306]:
df_mad_over100k=pd.concat([df_mad_40_60,df_mad_60_80,df_mad_80_100])

In [308]:
post_codes_spain=pd.read_csv('listado_codigos_postales_latlon.csv',sep=';')

In [309]:
post_codes_spain.columns

Index(['provincia', 'poblacion', 'codigopostalid', 'lat', 'lon'], dtype='object')

In [310]:
df_mad_over100k.columns

Index(['prices', 'post_code', 'rooms', 'bathrooms', 'size'], dtype='object')

In [311]:
post_codes_spain=post_codes_spain.drop(columns=['provincia'])

In [312]:
df_mad_over100k.post_code=df_mad_over100k.post_code.str.strip()

In [313]:
df_mad_over100k.loc[df_mad_over100k['post_code'].str.contains('a'),['post_code']]='0'
df_mad_over100k.loc[df_mad_over100k['post_code'].str.contains('e'),['post_code']]='0'
df_mad_over100k.loc[df_mad_over100k['post_code'].str.contains('i'),['post_code']]='0'
df_mad_over100k.loc[df_mad_over100k['post_code'].str.contains('o'),['post_code']]='0'
df_mad_over100k.loc[df_mad_over100k['post_code'].str.contains('u'),['post_code']]='0'
df_mad_over100k.loc[df_mad_over100k['post_code'].str.contains('á'),['post_code']]='0'
df_mad_over100k.loc[df_mad_over100k['post_code'].str.contains('é'),['post_code']]='0'
df_mad_over100k.loc[df_mad_over100k['post_code'].str.contains('í'),['post_code']]='0'
df_mad_over100k.loc[df_mad_over100k['post_code'].str.contains('ó'),['post_code']]='0'
df_mad_over100k.loc[df_mad_over100k['post_code'].str.contains('ú'),['post_code']]='0'

In [314]:
df_mad_over100k['post_code']=df_mad_over100k['post_code'].astype(int)

In [315]:
df_mad_over100k['post_code'].value_counts()

0        616
28025    321
28022    310
28004    260
28017    257
        ... 
28815      1
28936      1
28413      1
28794      1
28570      1
Name: post_code, Length: 180, dtype: int64

In [316]:
df_mad_over100k=df_mad_over100k.merge(post_codes_spain,left_on='post_code',right_on='codigopostalid')

In [318]:
df_mad_over100k.columns

Index(['prices', 'post_code', 'rooms', 'bathrooms', 'size', 'poblacion',
       'codigopostalid', 'lat', 'lon'],
      dtype='object')

In [319]:
df_mad_over100k['poblacion'].value_counts()

Madrid                   6767
Las Rozas de Madrid       135
Getafe                     99
Alcalá de Henares          88
Leganés                    68
                         ... 
Paracuellos de Jarama       1
Fresno de Torote            1
Orusco de Tajuña            1
Valdetorres de Jarama       1
Navacerrada                 1
Name: poblacion, Length: 82, dtype: int64

In [ ]:
df_mad_over100k

Test Scrapper for Trovit with only first 3 pages to see how it works and show first results in a DataFrame

In [217]:
#Cleaning lists:

list_prices=[]
list_features=[]
list_latitudes=[]
list_longitudes=[]
list_locations=[]

list_prices_full=[]
list_features_full=[]
list_latitudes_full=[]
list_longitudes_full=[]
list_locations_full=[]

#Scrapper with multiple for loops over the Trovit website

#First loop goes through each page and adds up the data to be able to build the DataFrame

for u in range (1,10):

    url='https://casas.trovit.es/index.php/cod.search_homes/type.1/what_d.madrid/origin.1/price_min.100000/rooms_min.1/photos.1/area_min.40/area_max.60/bathrooms_min.1/order_by.relevance/region.Madrid/resultsPerPage.25/isUserSearch.1/page.'
    driver.get(url+str(u))
    
    list_prices_full=list_prices_full+list_prices
    list_features_full=list_features_full+list_features
    list_latitudes_full=list_latitudes_full+list_latitudes
    list_longitudes_full=list_longitudes_full+list_longitudes
    list_locations_full=list_locations_full+list_locations

    locations=soup.find_all(attrs={'itemprop': 'address'})
    prices=soup.find_all('span',class_="amount")  
    features=soup.find_all('div' ,class_="property")
    latitudes=soup.find_all('meta',attrs={'itemprop': 'latitude'}, content=True)
    longitudes=soup.find_all('meta',attrs={'itemprop': 'longitude'}, content=True) 
    
    time.sleep(randint(1,4))
     
    #Second loop scrolls automatically to replicate a user behaviour:
    
    for i in range (8):
             
        html_txt=driver.page_source
        soup=BeautifulSoup(html_txt)  
        
        list_prices=[]
        list_locations=[]
        list_features=[]
        list_latitudes=[]
        list_longitudes=[]
        
        #This loops search for content in the page

        for price in prices:
            title1=price.getText()
            list_prices.append(title1)
        for feature in features:
            title2=feature.getText()
            list_features.append(title2)
        for location in locations:
            title3=location.getText()
            list_locations.append(title3)
        for latitude in latitudes:
            title4=latitude.get('content')
            list_latitudes.append(title4)
        for longitude in longitudes:
            title5=longitude.get('content')
            list_longitudes.append(title5)

        ActionChains(driver).key_down(Keys.PAGE_DOWN).key_up(Keys.PAGE_DOWN).perform()
        time.sleep(randint(1,10)/10)   
        

In [218]:
#Building the full DataFrame with all read data through the scrapper

list_locations_full= [i.split(',', 1)[0] for i in list_locations_full]
list_locations_full= [x.strip(' ') for x in list_locations_full]

list_features_full_splitted = [list_features_full[i*3:(i+1)*3] for i in range((len(list_features_full)+3-1)//3)] 
features_df= pd.DataFrame(list_features_full_splitted)
features_df.columns= ['rooms','bathrooms','size']

houses_list_full=[list_prices_full,list_locations_full,list_latitudes_full,list_longitudes_full]
result_trvt_df=pd.DataFrame()
result_trvt_df=pd.DataFrame (houses_list_full).transpose()
result_trvt_df.columns=['prices','post_code','latitude','longitude']
result_trvt_df= pd.concat([result_trvt_df, features_df], axis=1)  
result_trvt_df

,prices,post_code,latitude,longitude,rooms,bathrooms,size
0,184.000€,28012,40.41455339,-3.69499042,1 hab,1 wc,58 m²
1,116.900€,28037,40.42618850,-3.61629400,3 hab,1 wc,60 m²
2,109.200€,28038,40.39642662,-3.64093308,2 hab,1 wc,50 m²
3,116.900€,28037,40.42872662,-3.61591308,3 hab,1 wc,60 m²
4,112.062€,28031,40.37892762,-3.63187508,1 hab,1 wc,49 m²
5,230.000€,28012,40.41875082,-3.70286248,1 hab,1 wc,53 m²
6,122.400€,28019,40.38858759,-3.72159659,3 hab,1 wc,53 m²
7,135.000€,Comunidad de Madrid \n,40.42529662,-3.69696308,2 hab,1 wc,46 m²
8,350.000€,28004,40.35596962,-3.68370808,2 hab,1 wc,56 m²
9,133.100€,28021,40.40004380,-3.76772096,1 hab,1 wc,48 m²
